In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import mysql.connector
import datetime

# Conectarse a la base de datos
mydb = mysql.connector.connect(
    host="195.235.211.197",
    user="pc2_grupo5",
    password="Proyectos.23",
    database="pc2_grupo5"
)

# Crear un cursor para ejecutar las consultas SQL
cursor = mydb.cursor()


ejercicios = []
pasosdelejercicios = []
nombredelejercicio = []

    
msc = {}


body = ["chest","shoulders","traps","biceps","forearms","obliques","abdominals","quads","calves","lats","lowerback","glutes","hamstrings","traps_middle"]
bodyt=["chest"]

timestamp = datetime.datetime.now()

for mus in body:
    musculo_verificar = mus
    print(musculo_verificar)
    
    cursor.execute("SELECT id FROM muscles WHERE name = %s", (musculo_verificar,))
    result = cursor.fetchone()
    
    # Si el músculo no existe, insertarlo en la tabla "muscle"
    if not result:
        cursor.execute("INSERT INTO muscles (name, created_at) VALUES (%s, %s)", (musculo_verificar,timestamp))
        muscle_id = cursor.lastrowid
    else:
        muscle_id = result[0]  
    
    # URL de la página web a scrappear
    url = 'https://musclewiki.com/exercises/male/' + str(mus)

    # Realizar la petición HTTP y obtener el contenido de la página web
    response = requests.get(url)
    html = response.content

    # Crear el objeto BeautifulSoup y analizar el contenido HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Buscar los elementos que contienen los ejercicios por músculo
    msc[mus] = []
    
    muscles = soup.find_all('h3')
    descript = soup.find_all('ol')
    difficult = soup.find_all('p')
    videos = soup.find_all('video')
    
    videoss = []
    for video in videos:
        url = video['src']
        videoss.append(url)
    
    difficulties=[]
    for dif in difficult:
        d = dif.text.split()[1]
        difficulties.append(d)

    steps = []
    for desc in descript:
        descripciones = desc.find_all('li')
        muscle_steps = []  # Lista para almacenar los pasos del ejercicio actual
        for d in descripciones:
            muscle_steps.append(d.text)
        steps.append(muscle_steps)  # Agregar los pasos del ejercicio actual a la lista de pasos por músculo
    
    step_div = []
    for step in steps:
        step_div.append(step)
    
    i=0
    j=0
    
    
    
    # Recorrer cada músculo encontrado e imprimir los ejercicios correspondientes
    for muscle in muscles:
        # Buscar los enlaces de los ejercicios correspondientes a este músculo
        exercises = muscle.find_all('span',{'class':"mw-headline"})
        # Imprimir los enlaces de los ejercicios
        for exercise in exercises:

            ej = exercise.text.replace('\n', '').replace('Copied', '').strip()
             # Buscar si el ejercicio ya existe en la tabla "exercise" para el músculo actual
            cursor.execute("SELECT id FROM exercises WHERE name = %s AND muscle_id = %s", (ej, muscle_id))
            result = cursor.fetchone()
            

            
             # Si el ejercicio no existe, insertarlo en la tabla "exercise"
            if not result:
                cursor.execute("INSERT INTO exercises (name, difficulty, url, muscle_id, created_at) VALUES (%s, %s, %s, %s, %s)", 
                               (ej, difficulties[i], videoss[j], muscle_id, timestamp))
                exercise_id = cursor.lastrowid
            else:
                exercise_id = result[0]
            
            description = '\n'.join(step_div[i])
            #print(str(len(description)) + " - exercise_id:" + str(exercise_id))
            cursor.execute("INSERT INTO steps (description, exercise_id) VALUES (%s, %s)", (description, exercise_id))
            
            i += 1

#primero hacer un select del musculo a insertar (coger el id)

#luego hacer un select de todos los ejercicios que tiene  ese musclulo
    # si ya existe ese ejercico no lo inserto
    # si no existe dicho ejercicio lo inserto

#finalmente del ejercico hacer otro select para coger el id e insertar todos los pasos 

# Hacer commit de las transacciones y cerrar la conexión
mydb.commit()
cursor.close()
mydb.close()

# Iterar sobre los músculos y los ejercicios encontrados
